In [ ]:
# import

# Simple Model Run
##  xgBoost

In [61]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)

dtrain = xgb.DMatrix(df_train, label=df_y, enable_categorical=True)
dtest = xgb.DMatrix(df_test, enable_categorical=True)

params = {
    'max_depth': 6,
    'eta': 0.01,
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'mlogloss',
}
num_boost_round = 500

bst = xgb.train(params, dtrain, num_boost_round)
y_test_probs = bst.predict(dtest)

# Compute Multiclass Logarithmic Loss
#val_log_loss = log_loss(y_val, y_test_probs)
#print(f"Validation Multiclass Logarithmic Loss: {val_log_loss}")

class_order = [0, 1, 2, 3, 4]
class_mapping = {class_label: f"Class_{class_label}" for class_label in class_order}

submission_df = pd.DataFrame(y_test_probs, columns=class_mapping.values())
submission_df.columns = ['no answer', 'very important', 'quite important', 'not important', 'not at all important']
submission_df.insert(0, 'id', df_test.index)

# Save the submission file
submission_file = ('submission.csv')
submission_df.to_csv(submission_file, index=False)